In [ ]:
import sys
import time
from typing import final
import numpy as np

In [ ]:
str_board = "........W.W.W.W..w.w.w.w................b.b.b.b..B.B.B.B........"

In [ ]:
N = 8

In [ ]:
number_board = []
for i in range(N):
        number_board.append([p for p in str_board[N*i:N*(i+1)]]) 

In [ ]:
board = number_board

In [90]:
def print_board(board,N):
    for i in range(N):
        print("".join(board[i]))

In [ ]:
def state_to_string(board, N):
    return "".join("".join(board[i]) for i in range(N))

In [ ]:
print(state_to_string(board,N))

In [34]:
import copy
import math


def slopeAngle(p1,p2):   
    """Returns the minimum angle the line connecting two points makes with the x-axis.
    Args:
        p1 ([tuple]): point p1
        p2 ([tuple]): point p2
    Returns:
        [float]: minimum angle between x-axis and line segment joining p1 and p2
    """
    try :
        m = (p2[1]-p1[1])/(p2[0]-p1[0])
    except ZeroDivisionError:
        m = 0
    return abs(math.degrees(math.atan(m)))



def all_cells_in_bw(init,final):
    slope = slopeAngle(init,final)
      
    if slope in [0,90]:
        #points are in either same column or same row
        if init[0] == final[0]: #checking for same column
            points_between_them = [(final[0],i) for i in range(min(init[1],final[1])+1, max(init[1],final[1]))]

        elif init[1] == final[1]: #checking for same row
            points_between_them = [(i,final[1]) for i in range(min(init[0],final[0])+1, max(init[0],final[0]))]

        else: 
            raise ValueError("slope == 90 but no commanility between row and column; init = ("+", ".join(init)+")"+" final = ("+", ".join(final)+")"+" Slope: "+str(slope))

    elif slope == 45:
        #points are diagonal to each other
        m = (final[1]-init[1])/(final[0]-init[0])
        intercept = init[1] - init[0]*m

        x_points = [i for i in range(min(init[0],final[0])+1, max(init[0],final[0]))]
        points_between_them = [(x,int(intercept+m*x)) for x in x_points]

    else:
        raise("points are not in visible line of sight")
    
    return points_between_them

def nextPoint(init,final,N):
    m = slopeAngle(init,final) 
    if m in [0,90]:
        if final[1] == init[1]:
            point = ((int(final[0]+(final[0] - init[0])/abs((final[0] - init[0]))), final[1]))
        elif final[0] == init[0]:
            point = ((final[0], int(final[1]+(final[1] - init[1])/abs((final[1] - init[1]))) ))
    elif m in [45]:
        point = (((int(final[0]+(final[0] - init[0])/abs((final[0] - init[0])))), int(final[1]+(final[1] - init[1])/abs((final[1] - init[1]))) ))
    
    return point if valid_index(point,N,N) else None





def move_piece(board, player, piece, initial, final, jumpFlag=False, jumpPosition = None):
    # print("hjhgghj")
    # print(final)
    # print("ghghghghgh")

    # print("Moving Coin from init", initial , " to final", final )

    #alters the board
    assert ((jumpFlag) and (jumpPosition != None)) or ((not jumpFlag) and (jumpPosition == None)), "Jump position not passed but jump flag is on"

    #Checking for creation of Raichu
    if final[0] == len(board)-1:
        if player == "w":
            if player in "wW":
                piece = "@"
        elif player == "b":
            if player in "bB":
                piece = "$"

    temp = copy.deepcopy(board)
    temp[initial[0]][initial[1]] = "."

    if jumpFlag:
        temp[jumpPosition[0]][jumpPosition[1]] = "."


    temp[final[0]][final[1]] = piece

    return temp

def make_move(board,N,player,piece,init,final):
    #if valid move between init and final  and final is empty then moves the piece and returns board
    in_between = all_cells_in_bw(init,final)
    content = [board[pt[0]][pt[1]] for pt in in_between]
    
    # print("probable move from init", init , " to final", final )
    # if final == (1,4):
    #     print(init)
    #     print(final)
    #     print("in_bw",in_between)
    #     print("Con",content)
    #     print("board",(board[final[0]][final[1]] == "."))
    
    
    if (len(content) == 0) and  (board[final[0]][final[1]] == "."):
        #either no intersections and and final cell empty
        
        return move_piece(board=board, player=player, piece=piece, initial=init, final=final)

    elif (len(content) == 0) and (board[final[0]][final[1]] != "."):
        #no intersection but final cell is not empty 
        # check next cell in the same direction if empty and final cell content in capability then jump else immpossible move return None
        if board[final[0]][final[1]] in CAPABILITY[player][piece]:
            #check for next cell content
            destination = nextPoint(init,final,N) #position where the piece will be placed if vacant
            if destination and board[destination[0]][destination[1]] == ".":
                return move_piece(board=board, player=player, piece=piece, initial=init, final=destination, jumpFlag=True, jumpPosition = final)
            else:
                return None
        else:
            return None

    elif (len(content) != 0) and (board[final[0]][final[1]] != "."):
        #intersections present and final notcell  empty :
        #more thank one piece in path cannot jump 
        if (content.count(".") == len(content)) and (board[final[0]][final[1]] in CAPABILITY[player][piece]):
            destination = nextPoint(init,final,N) #position where the piece will be placed if vacant
            if destination and board[destination[0]][destination[1]] == ".":
                return move_piece(board=board, player=player, piece=piece, initial=init, final=destination, jumpFlag=True, jumpPosition = final)
            else:
                return None
        else:
            return None
    
    elif (len(content) != 0) and (board[final[0]][final[1]] == "."):
        #2 cases possible
        #one piece in intersection'
        if content.count(".") == len(content):
            return move_piece(board=board, player=player, piece=piece, initial=init, final=final)

        if content.count(".") == len(content)-1:
            #intersection piece in capability or not capability or same team piece 
            try:
                #handles if intersection piece is in capability 
                return move_piece(board=board, player=player, piece=piece, initial=init, final=final, jumpFlag=True, jumpPosition = in_between[[i for i,j in enumerate(content) if j != "." and j in CAPABILITY[player][piece]][0]])
            except IndexError:
                #either the intersection piece is out of capability or same team piece 
                return None 
                
        #more than one opposite piece in intersection or either one opposite piece and one same team piece  or 1 or more same team pieces implies no move
        else:
            return None


    elif (len(content) != 0) and (content.count(".") == len(content)) and (board[final[0]][final[1]] == "."):
        #there are intersections but all intersection empty and final cell is also empty
        return move_piece(board=board, player=player, piece=piece, initial=init, final=final)


    
    else:
        NotImplementedError("Case Missing in make_move")

In [ ]:
board

In [ ]:
d = make_move(board=board,N=N,player="w",piece="W",init=(5,1),final=(6,1))

In [ ]:
board_to_string(board,N)

In [ ]:
board[5][1] = ""
# board[6][3] = "B"
board[1][0] = "."
# board[5][2] = "b"
# board[6][5] = "W"
# board[5][6] = "."

# board [5][0] ='b' 
# board [4][1]  ="." 

In [21]:
import math
def slopeAngle(p1,p2):   
    """Returns the minimum angle the line connecting two points makes with the x-axis.
    Args:
        p1 ([tuple]): point p1
        p2 ([tuple]): point p2
    Returns:
        [float]: minimum angle between x-axis and line segment joining p1 and p2
    """
    try :
        m = (p2[1]-p1[1])/(p2[0]-p1[0])
    except ZeroDivisionError:
        m = 0
    return abs(math.degrees(math.atan(m)))

def all_cells_in_bw(init,final):
    slope = slopeAngle(init,final)
      
    if slope in [0,90]:
        #points are in either same column or same row
        if init[0] == final[0]: #checking for same column
            points_between_them = [(final[0],i) for i in range(min(init[1],final[1])+1, max(init[1],final[1]))]

        elif init[1] == final[1]: #checking for same row
            points_between_them = [(i,final[1]) for i in range(min(init[0],final[0])+1, max(init[0],final[0]))]

        else: 
            raise ValueError("slope == 90 but no commanility between row and column; init = ("+", ".join(init)+")"+" final = ("+", ".join(final)+")"+" Slope: "+str(slope))

    elif slope == 45:
        #points are diagonal to each other
        m = (final[1]-init[1])/(final[0]-init[0])
        intercept = init[1] - init[0]*m

        x_points = [i for i in range(min(init[0],final[0])+1, max(init[0],final[0]))]
        points_between_them = [(x,int(intercept+m*x)) for x in x_points]

    else:
        raise("points are not in visible line of sight")
    
    return points_between_them

In [ ]:
all_cells_in_bw((3,0),(5,0))

In [ ]:
content = [board[pt[0]][pt[1]] for pt in all_cells_in_bw((3,0),(5,0))]

In [ ]:
content

In [ ]:
final = (5,0)

In [ ]:
board[final[0]][final[1]] in CAPABILITY['w']['W']

In [20]:
CAPABILITY = {
    "w": {
        "w" : "b",
        "W": "bB",
        "@": "bB$" 
    } ,
    "b": {
        "b" : "w",
        "B": "wW",
        "$": "wW@"
    }
}

In [ ]:
content = ["."]

In [ ]:
board

In [ ]:
init = (4,0)

In [ ]:
final = (7,0)

In [ ]:
content = [board[pt[0]][pt[1]] for pt in all_cells_in_bw((4,0),(7,0))]

In [ ]:
try:
    all_cells_in_bw((4,0),(7,0))[[i for i,j in enumerate(content) if j != "." and j in CAPABILITY["w"]["W"]][0]]
except IndexError:
    None

In [ ]:
content[0] = "$"

In [ ]:
content

In [ ]:
board[4][0] = "W"

In [ ]:
board[3][0] = "."

In [ ]:
def genMoves(N):
    row,col = 0,0
    fw = [(row+i,col) for i in range(1,N)]
    bk = [(row-1,col) for i in range(1,N)]

    left = [(row,col-i) for i in range(1,N)]
    right = [(row,col+i) for i in range(1,N)]

    diag_lr_fw = [(i,i) for i in range(1,N)]
    diag_lr_bk = [(-i,-i) for i in range(1,N)]
    diag_rl_bk = [(-i,i) for i in range(1,N)]
    diag_rl_fw = [(i,-i) for i in range(1,N)]

    return { "pichu" : diag_lr_fw[0:1] + diag_rl_fw[0:1],
             "pikachu" : fw[0:2] + right[0:2] + left[0:2] }

ALL_MOVES = genMoves(N)


row,col = 3,0
for move in ALL_MOVES["pikachu"]:
        move = (move[0]+row,move[1]+col)
        if not valid_index(move, len(board), len(board[0])):
            continue
        print(move)

In [ ]:
all_cells_in_bw((1,0),(2,0))

In [ ]:
init  =(4,6)
final =(4,5)

def nextPoint(init,final,N):
    m = slopeAngle(init,final) 
    if m in [0,90]:
        if final[1] == init[1]:
            point  ((int(final[0]+(final[0] - init[0])/abs((final[0] - init[0]))), final[1]))
        elif final[0] == init[0]:
            point  ((final[0], int(final[1]+(final[1] - init[1])/abs((final[1] - init[1]))) ))
    elif m in [45]:
        point (((int(final[0]+(final[0] - init[0])/abs((final[0] - init[0])))), int(final[1]+(final[1] - init[1])/abs((final[1] - init[1]))) ))
    
    return point if valid_index(point,N,N) else None

In [ ]:
final[0] + ((final[1] - init[1])/abs((final[1] - init[1])))

In [ ]:
final

In [ ]:
"hg" if None else "jj"

In [ ]:
# Python code to demonstrate
# Count Strings with substring String List
# using filter() + lambda + len()

# initializing list
test_list = ['GeeksforGeeks', 'Geeky', 'Computers', 'Algorithms']

# printing original list
print ("The original list is : " + str(test_list))

# initializing substring
subs = 'Geek'

# using filter() + lambda + len()
# Count Strings with substring String List
res = list(filter(lambda x: subs in x, test_list))

# printing result
print(res)


In [ ]:
N= 8

In [ ]:
# def genMoves(N):
row,col = 0,0
fw = [(row+i,col) for i in range(1,N)]
bk = [(row-i,col) for i in range(1,N)]

left = [(row,col-i) for i in range(1,N)]
right = [(row,col+i) for i in range(1,N)]

diag_lr_fw = [(i,i) for i in range(1,N)]
diag_lr_bk = [(-i,-i) for i in range(1,N)]
diag_rl_bk = [(-i,i) for i in range(1,N)]
diag_rl_fw = [(i,-i) for i in range(1,N)]

#     return { "pichu" : diag_lr_fw[0:1] + diag_rl_fw[0:1],
#              "pikachu" : fw[0:2] + right[0:2] + left[0:2] }

In [ ]:
possible = fw[:] + right[:] + left[:] + bk[:] + diag_lr_bk[:] + diag_lr_fw[:] + diag_rl_bk[:] + diag_rl_fw[:]

In [ ]:
# move = (move[0]+row,move[1]+col)

row = 7
col = 1

[(move[0]+row,move[1]+col) for move in possible if valid_index((move[0]+row,move[1]+col),N,N)]

In [ ]:
??valid_index

In [ ]:
board

In [ ]:
p = ""
for i in d:
    p = p+"".join(i)
print (p)

In [ ]:
board

In [ ]:
d

In [ ]:
board

In [ ]:
import numpy as np

In [ ]:
fb = np.flip(board,0).tolist()

In [ ]:
fb 

In [ ]:
j = np.flip(fb,0).tolist()

In [ ]:
%%timeit
sum([j in "wW@" for i in board for j in i])

In [ ]:
fb

In [ ]:
p = ""
for i in j:
    p = p+"".join(i)
print (p)

In [ ]:
j

In [ ]:
board

In [ ]:
board[5][1]="."
board[6][1]="."
board[7][1]="W"

In [ ]:
def utility(board,main_player):
    white = sum([j in "wW@" for i in board for j in i])
    black = sum([j in "bB$" for i in board for j in i])
    # print(white,black)
    # if white-black == 3:
    #     print (white - black)
        # print("&&&&&&&&&&&&&&&&&")
        # print_board(board,len(board))
        # print("&&&&&&&&&&&&&&&&&")
    # return white-black   
    return white-black if main_player == "w" else black-white


utility(board,"w")

In [ ]:
........
..W.W.W.
.w.w.w.w
........
........
b.b.b.b.
...B.B.B
.W......

In [ ]:
def min_value(board,d, max_depth, player, main_player):
    nextPlayer = "b" if player == "w" else "w"
    if d == max_depth:
        return utility(board,main_player)
    else:
        value = np.inf
        for a in actions(board,player):
            value = min(value, max_value(a, d+1, max_depth, nextPlayer, main_player))
        return value

In [ ]:
def min_value(board, alpha, beta, d, max_depth, player):
    nextPlayer = "b" if player == "w" else "w"
    if d == max_depth:
        return utility(board)
    else:
        value = np.inf
        for a in actions(board,player):
            value = min(value, max_value(a, alpha, beta, d+1, max_depth, nextPlayer))
        return value

def max_value(board, alpha, beta, d, max_depth, player):
    nextPlayer = "b" if player == "w" else "w"
    if d == max_depth:
        return utility(board)
    else:
        value = -1*np.inf
        for a in actions(board,player):
            value = max(value, min_value(a, d+1, max_depth, nextPlayer))
        return value

In [12]:
def getCount(board):
    unique, counts = np.unique(np.array(board), return_counts=True)
    return dict(zip(unique,counts))
    
    
def weightedTypeDiff(board,counts,main_player):
    weights = [1,2,3]
    
    
    #Pichus
    w_pichu = 0 if "w" not in counts.keys() else counts["w"]
    b_pichu = 0 if "b" not in counts.keys() else counts["b"]
    pichu =  w_pichu-b_pichu if main_player == "w" else b_pichu-w_pichu
    
    #pikachus
    w_pikachu = 0 if "W" not in counts.keys() else counts["W"]
    b_pikachu = 0 if "B" not in counts.keys() else counts["B"]
    pikachu =  w_pikachu-b_pikachu if main_player == "w" else b_pikachu-w_pikachu
    
    #raichus
    w_raichu = 0 if "@" not in counts.keys() else counts["@"]
    b_raichu = 0 if "$" not in counts.keys() else counts["$"]
    raichu =  w_raichu-b_raichu if main_player == "w" else b_raichu-w_raichu
    
    return weights[0]*pichu + weights[1]*pikachu + weights[2]*raichu

def utility(board,main_player):
    #weight difference between types of white pieces and black pieces
    counts = getCount(board)
    return weightedTypeDiff(board,counts,main_player)

In [48]:
counts = getCount(board)

In [74]:
from scipy.spatial import distance
import numpy as np

def valid_index(pos, n, m):
        return 0 <= pos[0] < n  and 0 <= pos[1] < m

def nextPoint(init,final,N):
    m = slopeAngle(init,final) 
    if m in [0,90]:
        if final[1] == init[1]:
            point = ((int(final[0]+(final[0] - init[0])/abs((final[0] - init[0]))), final[1]))
        elif final[0] == init[0]:
            point = ((final[0], int(final[1]+(final[1] - init[1])/abs((final[1] - init[1]))) ))
    elif m in [45]:
        point = (((int(final[0]+(final[0] - init[0])/abs((final[0] - init[0])))), int(final[1]+(final[1] - init[1])/abs((final[1] - init[1]))) ))
    
    return point if valid_index(point,N,N) else None


def isThreat(a,b,threat_piece,N):
    max_range_info = {
        "w":1,"b":1,"W":2,"B":2,"$":np.inf,"@":np.inf
    }
    
    angle_info = {
        "w":[45],"b":[45],"W":[90,0],"B":[90,0],"$":[0,90,45],"@":[0,90,45]
    }
    
    
    if (slopeAngle(a,b) in angle_info[threat_piece]) and (distance.euclidean(a , b) <= max_range_info[threat_piece]):
        next_point = nextPoint(b,a,N)
        if next_point:
            if board[next_point[0]][next_point[1]] == ".":
                content = [board[pt[0]][pt[1]] for pt in all_cells_in_bw(b,next_point)]
                if len(content) == 0:
                    return True
                elif content.count(".") == len(content) - 1:
                    return True
        
    
    return False

def Threats(board, counts, player):
    state = np.array(board)
    player_info = {"w": "wW@","b":"bB$"}
    player_threat_info = {"b": "wW@","w":"bB$","W":"B$","B":"W@","@":"$","$":"@"}
    threat_weights = {"w":1,"W":2,"@":3,"b":1,"B":2,"$":3}
    threatSum = 0
    for each in player_info[player]:
        print(each)
        if each in counts.keys():
            for each_position in np.argwhere(state == each):
                for each_threat in player_threat_info[each]:
                    if each_threat in counts.keys():
                        for threat_position in np.argwhere(state == each_threat):
                            threatSum += threat_weights[each] if isThreat(each_position,threat_position,each_threat,len(board)) else 0

    return threatSum

def utility(board,main_player):
    #weight difference between types of white pieces and black pieces
    counts = getCount(board)
    return remaining(counts,main_player) + Threats(board,counts,"w" if main_player=="b" else "b") + (-1*Threats(board,counts,main_player))
#     return weightedTypeDiff(board,counts,main_player) + Threats(board,counts,"w" if main_player=="b" else "b") + (-1*Threats(board,counts,main_player))


In [75]:
def remaining(counts,player):
    remaining = 0
    player_info = {"w": "wW@","b":"bB$"}
    for each in player_info["w" if player=="b" else "b"]:
        if each in counts.keys():
            remaining+=counts[each]
    return -1*remaining

In [87]:
def distToRaichu(board,player):
    distance = 0
    state = np.array(board)
    player_info = {"w": "wW","b":"bB"}
    if player == "b":
        state = np.flip(state,0)
    for each in player_info[player]:
        for each_loc in np.argwhere(state == each):
            distance+= (len(board) - each_loc[0]) - 1
    return distance

In [88]:
distToRaichu(board,"b")

4

In [ ]:
....$.....b........b..........w................@.......B........

In [76]:
# utility(board,"w")
Threats(board,counts,"w")

w
[4 5] [4 4] ['w']
W
[6 4] [4 4] ['.', 'W']
@


<ipython-input-34-cbf3da8d1946>:14: RuntimeWarning: divide by zero encountered in longlong_scalars
  m = (p2[1]-p1[1])/(p2[0]-p1[0])


3

In [85]:
board

[['.', '.', '.', '.', '.', '.', '.', '.'],
 ['W', '.', '.', '.', 'W', '.', 'W', '.'],
 ['.', '.', '.', '.', '.', '.', '.', 'w'],
 ['.', '.', '.', '.', '.', '.', 'w', '.'],
 ['.', '.', '.', '.', 'B', 'w', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', 'W', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.']]

In [ ]:
for each_position in np.argwhere(a == "B"):
    print(each_position)

In [ ]:
%%timeit
np.argwhere(a == "w")

In [ ]:
%%timeit
np.argwhere(a == "b").tolist()

In [ ]:
np.argwhere(a == "B")

In [ ]:
np.argwhere(a == "$")

In [ ]:
np.argwhere(a == "@")

In [ ]:
utility(board,"w")

In [49]:
main_player

NameError: name 'main_player' is not defined

In [7]:
........W...W.W........w......w.....Bw..............W...........

SyntaxError: invalid syntax (<ipython-input-7-1d6bd9296f3d>, line 1)

In [8]:
str_board = "........W...W.W........w......w.....Bw..............W..........."
number_board = []
for i in range(N):
        number_board.append([p for p in str_board[N*i:N*(i+1)]])
board = number_board 

In [9]:
board

[['.', '.', '.', '.', '.', '.', '.', '.'],
 ['W', '.', '.', '.', 'W', '.', 'W', '.'],
 ['.', '.', '.', '.', '.', '.', '.', 'w'],
 ['.', '.', '.', '.', '.', '.', 'w', '.'],
 ['.', '.', '.', '.', 'B', 'w', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', 'W', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.']]

In [ ]:
%%timeit


In [ ]:
unique

In [ ]:
counts

In [ ]:
.........BB......B.B................@....................$......

In [ ]:
str_board = ".........BB......B.B....................................@$......"
number_board = []
for i in range(N):
        number_board.append([p for p in str_board[N*i:N*(i+1)]])
board = number_board 

In [ ]:
board

In [ ]:
str_board = "....$.....b........b..........w................................@"
number_board = []
for i in range(N):
        number_board.append([p for p in str_board[N*i:N*(i+1)]])
board = number_board 

In [ ]:
board

In [112]:
#### Utility Function and Supporting Functions

def getCount(board):
    unique, counts = np.unique(np.array(board), return_counts=True)
    return dict(zip(unique,counts))

def remaining(counts,player):
    remaining = 0
    player_info = {"w": "wW@","b":"bB$"}
    for each in player_info["w" if player=="b" else "b"]:
        if each in counts.keys():
            remaining+=counts[each]
    return remaining

def weightedTypeDiff(board,counts,main_player):
    weights = [1,2,3]
    
    
    #Pichus
    w_pichu = 0 if "w" not in counts.keys() else counts["w"]
    b_pichu = 0 if "b" not in counts.keys() else counts["b"]
    pichu =  w_pichu-b_pichu if main_player == "w" else b_pichu-w_pichu
    
    #pikachus
    w_pikachu = 0 if "W" not in counts.keys() else counts["W"]
    b_pikachu = 0 if "B" not in counts.keys() else counts["B"]
    pikachu =  w_pikachu-b_pikachu if main_player == "w" else b_pikachu-w_pikachu
    
    #raichus
    w_raichu = 0 if "@" not in counts.keys() else counts["@"]
    b_raichu = 0 if "$" not in counts.keys() else counts["$"]
    raichu =  w_raichu-b_raichu if main_player == "w" else b_raichu-w_raichu
    
    return weights[0]*pichu + weights[1]*pikachu + weights[2]*raichu

def isThreat(board, a,b,threat_piece,N):
    max_range_info = {
        "w":1,"b":1,"W":2,"B":2,"$":np.inf,"@":np.inf
    }
    
    angle_info = {
        "w":[45],"b":[45],"W":[90,0],"B":[90,0],"$":[0,90,45],"@":[0,90,45]
    }
    
    
    if (slopeAngle(a,b) in angle_info[threat_piece]) and (distance.euclidean(a , b) <= max_range_info[threat_piece]):
        next_point = nextPoint(b,a,N)
        if next_point != None:
            if board[next_point[0]][next_point[1]] == ".":
                content = [board[pt[0]][pt[1]] for pt in all_cells_in_bw(b,next_point)]
                if len(content) == 0:
                    return True
                elif content.count(".") == len(content) - 1:
                    return True
        
    
    return False

def distToRaichu(board,player):
    distance = 0
    state = np.array(board)
    player_info = {"w": "wW","b":"bB"}
    if player == "b":
        state = np.flip(state,0)
    for each in player_info[player]:
        for each_loc in np.argwhere(state == each):
            distance+= (len(board) - each_loc[0]) - 1
    return distance


def Threats(board, counts, player):
    state = np.array(board)
    player_info = {"w": "wW@","b":"bB$"}
    player_threat_info = {"b": "wW@","w":"bB$","W":"B$","B":"W@","@":"$","$":"@"}
    threat_weights = {"w":1,"W":2,"@":3,"b":1,"B":2,"$":3}
    threatSum = 0
    for each in player_info[player]:
        if each in counts.keys():
            for each_position in np.argwhere(state == each):
                for each_threat in player_threat_info[each]:
                    if each_threat in counts.keys():
                        for threat_position in np.argwhere(state == each_threat):
                            threatSum += threat_weights[each] if isThreat(board, each_position,threat_position,each_threat, len(board)) else 0

    return threatSum

def utility(board,main_player, inital_remaining_my_pieces,inital_remaining_enemies):
    #weight difference between types of white pieces and black pieces
    counts = getCount(board)
    # return (weightedTypeDiff(board,counts,main_player) + # () +
    #         (1.5*Threats(board,counts,"w" if main_player=="b" else "b")) + 
    #         (-1*Threats(board,counts,main_player)))
    
    print("wd", (weightedTypeDiff(board,counts,main_player)))
    print("e_rm", (2*(inital_remaining_enemies - remaining(counts,main_player))))
    print("my_rem", (-1*(inital_remaining_my_pieces - remaining(getCount(board),"b" if main_player == "w" else "w"))))
    print("threat", (-1*Threats(board,counts,main_player)))
    print("distToRaichu", -1.5*distToRaichu(board,main_player))      
    return ((weightedTypeDiff(board,counts,main_player) + 
            2*(inital_remaining_enemies - remaining(counts,main_player)) +
            -1*(inital_remaining_my_pieces - remaining(getCount(board),"b" if main_player == "w" else "w")) + # (1.5* Threats(board,counts,"w" if main_player=="b" else "b")) + 
            (-1*Threats(board,counts,main_player))) -- 1.5*distToRaichu(board,main_player))

In [113]:
str_board = "....$.....b........b..........w................................@"
number_board = []
for i in range(N):
        number_board.append([p for p in str_board[N*i:N*(i+1)]])
board = number_board 
print_board(board, N)

utility(board,"w", 2, 4)

....$...
..b.....
...b....
......w.
........
........
........
.......@
wd -1
e_rm 2
my_rem 0
threat 0
distToRaichu -6.0


7.0

In [116]:
str_board = "..@.$.....b........b..........w........................B........"
number_board = []
for i in range(N):
        number_board.append([p for p in str_board[N*i:N*(i+1)]])
board = number_board 
print_board(board, N)

utility(board,"w", 2, 4)

..@.$...
..b.....
...b....
......w.
........
........
.......B
........
wd -3
e_rm 0
my_rem 0
threat -3
distToRaichu -6.0


<ipython-input-34-cbf3da8d1946>:14: RuntimeWarning: divide by zero encountered in longlong_scalars
  m = (p2[1]-p1[1])/(p2[0]-p1[0])


0.0